# Analisis Sentimen Ulasan Pariwisata Bali

## 1. Import Library

In [ ]:

import pandas as pd
import numpy as np

## 2. Data Loading

In [ ]:
base_path = r'C:\Users\Asus\Desktop\capstone\training_data\data'
locations_map = {
    'monkey_forest_reviews.csv': 'Monkey Forest',
    'tanah_lot_reviews.csv': 'Tanah Lot',
    'uluwatu_reviews.csv': 'Uluwatu'
}


## 3. Prepprocessing

## 4. Pelabelan Sentimen

## 5. Data Splitting

## 6. Modelling

## 7. Evaluasi Model

## 8. Visualisasi Data